# Statistical Modeling  With Python#
### Models for Continuous and Categorical Outcomes

This material uses Python to demonstrate some aspects of statistical models with continuous values to be predicted.

* If the dependent variable (the variable we are trying to explain or predict) is continuous (has a large range, like price of housing), then we use **multiple regression**.

* If the dependent variable is categorical (or represents a choice outcome) with two values (rent or own for example) than a **binary logit model** or **logistic regression** is usually the preferred model.

* if the dependent variable is categorical (or represents a choice outcome) with a small number of values (like travel modes), then the most common model form is **Multinomial Logit (MNL)**

In today's session we introduce Python libraries that enable estimating models the multiple regression type.  Theory is only briefly reviewed, as these methods should be supported by a full course (or more).  The material is really meant to help students who have been exposed to these methods using another platform such as Stata, R, SAS, or SPSS, and want to be able to use Python to undertake their statistical model building.

## Simple Linear Regression

Simple linear regression is an approach for predicting a **quantitative response** using a **single feature** (or "predictor" or "input variable"). It takes the following form:

$Y = \beta_0 + \beta_1x + \epsilon$, where $\epsilon\sim N\left(0,\sigma^{2}\right)$

- $y$ is the dependent variable - the one we are trying to explain or predict
- $x$ is the independent or explanatory variable that we are using to help explain or predict the value of $y$ with
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for x, which is the slope of the line that minimizes the sum of the squared errors
- $\epsilon$ is the error term, assumed to be normally distributed with mean of zero and variance of $\sigma^{2}$

Together, $\beta_0$ and $\beta_1$ are called the **model coefficients**. To create your model, you must "learn" or "estimate" the values of these coefficients. And once we've learned these coefficients, we can use the model to predict values of $y$ based on new values of $x$.

## Estimating Model Coefficients

Generally speaking, coefficients are estimated using the **least squares criterion**, which means we are find the line (mathematically) which minimizes the **sum of squared residuals** (or "sum of squared errors"):

In the figure below:
- The black dots are the **observed values** of x and y.
- The red line is our **least squares line**.
- The **residuals** are the vertical distances between the observed values and the least squares line.

How do the model coefficients relate to the least squares line?
- $\beta_0$ is the **intercept** (the value of $y$ when $x$=0)
- $\beta_1$ is the **slope** (the change in $y$ divided by change in $x$)

Here is a graphical depiction of those calculations:

<img src="regression.png">

In [ ]:
# Startup steps
import pandas as pd, numpy as np, statsmodels.api as sm
import matplotlib.pyplot as plt, matplotlib.cm as cm, matplotlib.font_manager as fm
import matplotlib.mlab as mlab
import time, requests
from scipy.stats import pearsonr, ttest_rel
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
import folium
from folium import plugins
from folium.plugins import MarkerCluster
from folium.map import FeatureGroup, Icon, Layer, Marker
from folium.plugins.measure_control import MeasureControl
%matplotlib inline

We begin by generating some synthetic data that is generated using an equation for which we supply the parameters.  It enables us to verify that the model estimation code is correctly 'learning' the correct parameters, before we use it on real data. 

Below we generate 100 values of Y from an equation: $Y = \beta_0 + \beta_1 x + \epsilon$.  

We set $\beta_0 = 0$ and $\beta_1 = 2$ and draw values of $\epsilon$ from a normal distribution

In [ ]:
nsample = 300
x = np.linspace(0, 10, nsample)
beta = np.array([0, 2])
e = np.random.normal(size=nsample)*2
X = sm.add_constant(x)
y = np.dot(X, beta) + e

Plot the data and the model.  Note that the intercept is set to zero in this example initially.

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.plot([0, 10], [0, 20])
plt.scatter(x, y, marker=0, s=10, c='g')
plt.axis([0, 10, 0, 20])
plt.show();

Now we 'fit' the model to the data, which means we compute the values of $\beta_0$ and $\beta_1$ that minimize the sum of the squared errors.  We use Statsmodels for fitting the model.  There are two different syntax styles to use for this.  The first one uses an X matrix and a y array, like this:

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

We can extract any of the results from the fitted model we want, since the fitted model is a Python object and has attributes we can interrogate.

In [ ]:
print('Parameters: ', results.params)

Statsmodels calculates 95% confidence intervals for our model coefficients, which are interpreted as follows: If the population from which this sample was drawn was **sampled 100 times**, approximately **95 of those confidence intervals** would contain the "true" coefficient.

To get the 95% confidence interval around $\beta_0$ and $\beta_1$ we can do this:

In [ ]:
results.conf_int()

## Hypothesis Testing and p-values

Closely related to confidence intervals is **hypothesis testing**. Generally speaking, you start with a **null hypothesis** and an **alternative hypothesis** (that is opposite the null). Then, you check whether the data supports **rejecting the null hypothesis** or **failing to reject the null hypothesis**.

(Note that "failing to reject" the null is not the same as "accepting" the null hypothesis. The alternative hypothesis may indeed be true, except that you just don't have enough data to show that.)

As it relates to model coefficients, here is the conventional hypothesis test:
- **null hypothesis:** There is no relationship between $x$ and $y$ (and thus $\beta_1$ equals zero)
- **alternative hypothesis:** There is a relationship between $x$ and $y$ (and thus $\beta_1$ is not equal to zero)

How do we test this hypothesis? Intuitively, we reject the null (and thus believe the alternative) if the 95% confidence interval **does not include zero**. Conversely, the **p-value** represents the probability that the coefficient is actually zero:

In [ ]:
results.pvalues

If the 95% confidence interval **includes zero**, the p-value for that coefficient will be **greater than 0.05**. If the 95% confidence interval **does not include zero**, the p-value will be **less than 0.05**. Thus, a p-value less than 0.05 is one way to decide whether there is likely a relationship between the feature and the response. (Again, using 0.05 as the cutoff is just a convention.)

In this case, the p-value for $x$ is far less than 0.05, and so we **believe** that there is a relationship between $x$ and $y$.

Note that we generally ignore the p-value for the intercept.

## How Well Does the Model Fit the data?

The most common way to evaluate the overall fit of a linear model is by the **R-squared** value. R-squared is the **proportion of variance explained**, meaning the proportion of variance in the observed data that is explained by the model, or the reduction in error over the **null model**. (The null model just predicts the mean of the observed response, and thus it has an intercept and no slope.)

R-squared is between 0 and 1, and higher is better because it means that more variance is explained by the model. 

In [ ]:
print('R2: ', results.rsquared)

We can also inspect the residuals (the errors) computed by comparing the predicted values of y to the observed ones.  The mean of the residuals should be zero and they should be normally distributed.

In [ ]:
results.resid.mean()

In [ ]:
from scipy.stats import norm
plt.rcParams['figure.figsize']=8,8
#plot(residuals.mean(),0, residuals.mean(), 2.25)
sns.set_style("white")
sns.set_style("ticks")
ax = sns.distplot(results.resid, fit=norm, kde=False)

## Multiple Linear Regression

Simple linear regression can easily be extended to include multiple features. This is called **multiple linear regression**:

$y = \beta_0 + \beta_1x_1 + ... + \beta_nx_n + \epsilon$

Let's add an additional x variable to our synthetic model, using x squared, and fit it again with Statsmodels.

We generate new values of $y$ using $\beta_0 = 0$, $\beta_1 = 2$ and $\beta_2 = 0.5$

In [ ]:
nsample = 300
x = np.linspace(0, 10, 300)
X = np.column_stack((x, x**2))
beta = np.array([1, 2, .5])
e = np.random.normal(size=nsample)*2

In [ ]:
X = sm.add_constant(X)
y = np.dot(X, beta) + e

In [ ]:
X[:10]

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

In [ ]:
ax = sns.regplot(x=X[:,1], y=y, scatter_kws={"s": 10}, order=2, ci=None, truncate=True)

In [ ]:
results.resid.mean()

In [ ]:
from scipy.stats import norm
plt.rcParams['figure.figsize']=8,8
sns.set_style("white")
sns.set_style("ticks")
ax = sns.distplot(results.resid, fit=norm, kde=False)

**R-squared will always increase as you add more features to the model**, even if they are unrelated to the response. Thus, selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.

There is alternative to R-squared called **adjusted R-squared** that penalizes model complexity (to control for overfitting), but it generally [under-penalizes complexity](http://scott.fortmann-roe.com/docs/MeasuringError.html).

## Estimating a Multiple Regression on Housing Prices (Hedonic Regression)##

Now let's use real data.  We pull some housing sales transactions from Redfin for this, for one month of sales in San Francisco, looking back from March 5th 2017.

In [ ]:
sf0 = pd.read_csv('data/redfin_2017-03-05-17-45-34-san-francisco-county-1-month.csv')
sf0.columns

In [ ]:
sf = sf0.rename(index=str, columns={'SALE TYPE': 'saletype',
    'SOLD DATE': 'solddate', 'PROPERTY TYPE': 'proptype', 'ADDRESS': 'address',
    'CITY': 'city', 'STATE': 'state', 'ZIP': 'zip', 'PRICE': 'price', 'BEDS': 'beds',
    'BATHS': 'baths', 'LOCATION': 'location', 'SQUARE FEET': 'sqft', 'LOT SIZE': 'lotsize',
    'YEAR BUILT': 'yrbuilt', 'DAYS ON MARKET': 'daysonmkt', '$/SQUARE FEET': 'pricesqft',
    'LATITUDE': 'latitude', 'LONGITUDE': 'longitude', 'HOA/MONTH': 'hoamonth',
    'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)': 'url',
    'STATUS': 'status', 'NEXT OPEN HOUSE START TIME': 'nextopenstart', 'NEXT OPEN HOUSE END TIME': 'nextopenend',
    'SOURCE': 'source', 'MLS#': 'mls', 'FAVORITE': 'favorite', 'INTERESTED': 'interested'
    })

sf.head()

In [ ]:
sf.columns

In [ ]:
sf.describe()

Let's get some additional census data to merge on to our sales transactions.  But we don't have a census block on our records.... what to do?  We can go to our trusty FCC geocoder API, of course!

**In the interest of time (it takes a couple of minutes to run), you can skip the next three cells and just start from the cell that loads the data from a csv 4 cells below.**

In [ ]:
def get_fips(row):
    time.sleep(pause)
    url = 'http://geo.fcc.gov/api/census/block/find?format=json&latitude={}&longitude={}'
    request = url.format(row['latitude'], row['longitude'])
    response = requests.get(request)
    data = response.json()
    
    # return multiple values as a series - this will create a dataframe with multiple columns
    return pd.Series({'fips_code':data['Block']['FIPS'], 'county':data['County']['name']})

In [ ]:
%%time
pause = 0.01
fips = sf.apply(get_fips, axis=1)
sf = pd.concat([sf, fips], axis=1)

In [ ]:
sf.head()

In [ ]:
sf.to_csv('data/sf1_blk.csv')

**Start from here if you want to skip the geocoding step...**

In [ ]:
sf = pd.read_csv('data/sf1_blk.csv', usecols=['proptype', 'price', 'beds', 'baths', 'sqft', 'lotsize',\
    'yrbuilt', 'hoamonth', 'fips_code', 'latitude', 'longitude'], converters={'fips_code': str})
sf.head()

In [ ]:
sv = pd.read_csv('data/smallvars.csv', converters={'block_id': str})
sv.head()

In [ ]:
sf1 = pd.merge(sf, sv, left_on='fips_code', right_on='block_id', how='inner')
sf1.columns

In [ ]:
g = sns.jointplot("sqft", "price", data=sf1, kind="reg", scatter_kws={"s": 10}, size=10)

Let's fit a simple linear regression of price on sqft, since there is clearly a strong relationship between them.

We will use Statsmodels for this as before, but now we use the Patsy syntax to specify the model.  Patsy uses the same format as the R language for specifying models.  The link below provides good documentation on the format.

http://www.statsmodels.org/dev/example_formulas.html

In [ ]:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices
y, X = dmatrices('price ~ sqft', 
                 data=sf1, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

How would you intepret the R-squared value?

Check-in time: http://bitly.com/cp255
How can you interpret the coefficient on sqft?

A common thing to explore is whether a transformation of the dependent and/or independent variables help improve the degree to which the relationship is linear, and the fit of the model.  Most models of housing prices, or 'hedonic price models' are specified with the log of price, and often the log of continuous variables like sqft.  Let's look at the relationship once we log-transform both:

In [ ]:
g = sns.jointplot(np.log(sf1['sqft']), np.log(sf1['price']), kind="reg", scatter_kws={"s": 8}, size=10)

And now let's re-estimate the model using the log-log transformation of price and sqft.

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft)', 
                 data=sf1, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

The model fit improves considerably.  How do you interpret the coefficient for sqft now?

Next let's add the number of baths and see how that changes the model.

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths', 
                 data=sf1, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

Let's add bedrooms now and see how that changes the model...

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + beds', 
                 data=sf1, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

Hmm.  Expected that result on bedrooms?

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + beds + np.log(lotsize)', 
                 data=sf1, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

Hmm... the R-Squared went down when we added a variable?  That's strange.  What happened?

In [ ]:
sf1.proptype.value_counts()

In [ ]:
single = sf1[sf1['proptype']=='Single Family Residential'] 

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + beds + np.log(lotsize) ', 
                 data=single, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

In [ ]:
condo = sf1[sf1['proptype']=='Condo/Co-op'] 

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + beds ', 
                 data=condo, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

**Adding dummy variables**

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + C(beds)  ', 
                 data=condo, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

**Adding Interaction Terms**

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + beds + beds * baths ', 
                 data=condo, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

**Adding neighborhood variables**

In [ ]:
y, X = dmatrices('np.log(price) ~ np.log(sqft) + baths + beds  + bgmedinc +\
                 proprent + lnjobs5000m', 
                 data=condo, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
residuals = res.resid
predicted = res.fittedvalues
observed = y
print(res.summary())

In [ ]:
data = observed.join(predicted.to_frame(name='predicted'))
data.head()

In [ ]:
plt.rcParams['figure.figsize']=8,8

sns.set_style("white")
sns.set_style("ticks")
ax = sns.distplot(residuals, fit=norm, kde=False)


In [ ]:
g = sns.jointplot(predicted, residuals, kind="reg", scatter_kws={"s": 8}, size=10)

In [ ]:
g = sns.jointplot("np.log(price)", "predicted", data=data, kind="reg", scatter_kws={"s": 8}, size=10)

If you want to explore your data further to look at pairwise relationships, you can use the Seaborn PairGrid plot.

In [ ]:
keepcols = [ 'price', 'beds', 'baths', 'sqft', 'lotsize', 'yrbuilt']
condo_small=condo[keepcols]
condo_small.head()

In [ ]:
g = sns.PairGrid(condo_small)
g.map(plt.scatter);


# Your turn

* Try experimenting with the model for single family or condo sales to see if you can improve it significantly using the available variables.

* If you have time and energy, you could try grabbing your own Redfin sales transactions for a place you are interested in. But keep in mind you'll need to add locational context variables on your own.  Census data is pretty easy, but accessibility variables require more work, using pandana and/or urbanaccess, and getting your own point of interest (POI) data to compute accessibility to.